Loding data

In [82]:
import json

params = {}
with open("proj4_params.json", "r", encoding="utf-8") as file:
    params = json.load(file)

In [83]:
try:
    import geopandas
except Exception:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "geopandas"])

gdf = geopandas.read_file("proj4_points.geojson")
print(gdf)

      lamp_id circuit         label                         geometry
0        5907    4260    4260 III/9  POINT (2215161.628 6459108.749)
1        5908    4104      4104 I/7   POINT (2214299.747 6459283.83)
2        5909    4070      4070 I/8  POINT (2215058.617 6458937.362)
3        5910    4148     4148 I/14   POINT (2214001.935 6459017.98)
4        5911    4148      4148 I/4   POINT (2214331.397 6458976.29)
...       ...     ...           ...                              ...
3737     9644    1025  1025 VIII/21  POINT (2219072.963 6459005.128)
3738     9645    1145     1145 I/11  POINT (2219170.783 6458492.182)
3739     9646    1145     1145 I/12  POINT (2219192.738 6458462.898)
3740     9647    1145     1145 I/13  POINT (2219214.247 6458434.278)
3741     9648    1146     1146 IV/1  POINT (2219070.984 6457949.465)

[3742 rows x 4 columns]


In [84]:
gdf_buffor = gdf.copy()
gdf_copy = gdf.copy()
gdf_copy = gdf_copy.to_crs(epsg=2180)
gdf_buffor = gdf_buffor.to_crs(epsg=2180)
gdf_buffor["geometry"] = gdf_buffor["geometry"].buffer(100)

joined = geopandas.sjoin(gdf_buffor, gdf_copy, how="left", predicate="intersects")
print(joined)

      lamp_id_left circuit_left  label_left  \
0             5907         4260  4260 III/9   
0             5907         4260  4260 III/9   
0             5907         4260  4260 III/9   
0             5907         4260  4260 III/9   
0             5907         4260  4260 III/9   
...            ...          ...         ...   
3741          9648         1146   1146 IV/1   
3741          9648         1146   1146 IV/1   
3741          9648         1146   1146 IV/1   
3741          9648         1146   1146 IV/1   
3741          9648         1146   1146 IV/1   

                                               geometry  index_right  \
0     POLYGON ((564419.617 245585.559, 564419.135 24...          685   
0     POLYGON ((564419.617 245585.559, 564419.135 24...          684   
0     POLYGON ((564419.617 245585.559, 564419.135 24...          683   
0     POLYGON ((564419.617 245585.559, 564419.135 24...          686   
0     POLYGON ((564419.617 245585.559, 564419.135 24...          687   
...

In [85]:
id_col = params["id_column"]
count_series = joined.groupby(id_col + "_left").size()
result = count_series.reset_index()
result.columns = [id_col, "count"]
print(result)
result.to_csv("proj4_ex01_counts.csv", index=False)

      lamp_id  count
0        5907     16
1        5908     16
2        5909     17
3        5910     20
4        5911      9
...       ...    ...
3737     9644     16
3738     9645     16
3739     9646     15
3740     9647     12
3741     9648     19

[3742 rows x 2 columns]


In [86]:
gdf_latlon = gdf.copy()
if gdf_latlon.crs != "EPSG:4326":
    gdf_latlon = gdf_latlon.to_crs(epsg=4326)

gdf_latlon['lat'] = gdf_latlon.geometry.y
gdf_latlon['lon'] = gdf_latlon.geometry.x

id_col = params["id_column"]
gdf_latlon[[id_col, 'lat', 'lon']].round({'lat': 7, 'lon': 7}).to_csv("proj4_ex01_coords.csv", index=False)

print(gdf_latlon[[id_col, 'lat', 'lon']].head())

   lamp_id        lat        lon
0     5907  50.074043  19.899135
1     5908  50.075053  19.891393
2     5909  50.073055  19.898210
3     5910  50.073520  19.888718
4     5911  50.073280  19.891677


In [98]:
city_name = params["city"]

try:
    import osmnx as ox
except Exception:
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "osmnx"])
    import osmnx as ox

gdf_roads = ox.features.features_from_place(query=city_name, tags={"highway": ["tertiary"]})

In [ ]:
gdf_roads = gdf_roads.reset_index()

gdf_roads["osm_id"] = gdf_roads["id"]

columns_to_keep = ["osm_id", "name", "geometry"]

gdf_roads = gdf_roads[columns_to_keep].dropna(subset=["geometry"])

gdf_roads.to_file("proj4_ex02_roads.geojson", driver="GeoJSON")


In [131]:
gdf_copy = gdf.copy().to_crs(epsg=2180)
gdf_roads_copy = gdf_roads.copy().to_crs(epsg=2180)

gdf_roads_copy["geometry"] = gdf_roads_copy.buffer(50, cap_style=2)

joined = geopandas.sjoin(gdf_copy, gdf_roads_copy, how="inner", predicate="within")

result = joined.groupby("name").size().reset_index(name="point_count")

result = result[result["point_count"] > 0]

result.to_csv("proj4_ex03_streets_points.csv", index=False)

print(result)

                                name  point_count
0                       Aleja 3 Maja          131
1                     Aleja Kijowska          128
2   Aleja Marszałka Ferdinanda Focha           33
3                            Balicka           79
4               Bartosza Głowackiego           23
5                         Bronowicka          148
6                      Czarnowiejska           14
7                     Dolnych Młynów           12
8                  Józefa Szujskiego            5
9                         Karmelicka           64
10              Kazimierza Wielkiego           66
11                         Krupnicza           23
12                         Królewska          121
13                  Królowej Jadwigi            4
14                    Księcia Józefa            8
15                     Lucjana Rydla           34
16     Marszałka Józefa Piłsudskiego           16
17                         Na Błonie            3
18                        Piastowska           61


In [136]:
gdf_countries = geopandas.read_file("proj4_countries.geojson")

import matplotlib.pyplot as plt

gdf_countries['geometry'] = gdf_countries['geometry'].boundary

gdf_countries = gdf_countries.to_crs(epsg=3857)

gdf_countries.to_pickle("proj4_ex04_gdf.pkl")

def plot_country_boundary(country_name, country_gdf_countries):
    fig, ax = plt.subplots(figsize=(10, 10))
    
    country_gdf_countries.plot(ax=ax, edgecolor='black', facecolor='none')
    
    ax.set_title(f"Boundary of {country_name.capitalize()}")
    ax.set_axis_off()
    
    output_file =f"proj4_ex04_{country_name.lower()}.png"
    plt.savefig(output_file, format="png", bbox_inches="tight", dpi=300)
    plt.close()

for idx, row in gdf_countries.iterrows():
    country_name = row['name']
    country_gdf_countries = gdf_countries[gdf_countries['name'] == country_name]
    
    plot_country_boundary(country_name, country_gdf_countries)